In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import fpgrowth
import pysubgroup as ps

df = pd.read_csv('./data/processed.csv')
df.head()

,Duration (in seconds),Q2,Q3,Q4,Q5,Q6_Coursera,Q6_edX,Q6_Kaggle Learn Courses,Q6_DataCamp,Q6_Fast.ai,...,"Q44_Reddit (r/machinelearning, etc)","Q44_Kaggle (notebooks, forums, etc)","Q44_Course Forums (forums.fast.ai, Coursera forums, etc)","Q44_YouTube (Kaggle YouTube, Cloud AI Adventures, etc)","Q44_Podcasts (Chai Time Data Science, O’Reilly Data Show, etc)","Q44_Blogs (Towards Data Science, Analytics Vidhya, etc)","Q44_Journal Publications (peer-reviewed journals, conference proceedings, etc)","Q44_Slack Communities (ods.ai, kagglenoobs, etc)",Q44_None,Q44_Other
0,121,30-34,Man,India,No,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,462,30-34,Man,Algeria,No,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,293,18-21,Man,Egypt,Yes,1,1,0,1,0,...,0,1,0,1,1,0,0,0,0,0
3,851,55-59,Man,France,No,1,0,1,0,0,...,0,1,1,0,0,1,0,0,0,0
4,232,45-49,Man,India,Yes,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
def filter_df(df, questions):
    cols = [*df]
    selected = []
    for q in questions:
        #Get associated columns
        selected.extend([c for c in cols if c == q or c.startswith(q + '_')])
    return df[selected]

filtered = filter_df(df, ['Q4', 'Q8', 'Q24', 'Q29','Q42'])
#filtered = filter_df(df, ['Q2', 'Q3', 'Q4','Q5'])
filtered.head()

,Q4,Q8,Q13_JupyterLab,Q13_RStudio,Q13_Visual Studio,Q13_Visual Studio Code (VSCode),Q13_PyCharm,Q13_Spyder,Q13_Notepad++,Q13_Sublime Text,...,Q29,Q42_GPUs,Q42_TPUs,Q42_IPUs,Q42_RDUs,Q42_WSEs,Q42_Trainium Chips,Q42_Inferentia Chips,Q42_None,Q42_Other
0,India,NaN,0,0,0,0,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
1,Algeria,Master’s degree,0,0,0,0,0,0,1,0,...,NaN,0,0,0,0,0,0,0,0,0
2,Egypt,Bachelor’s degree,1,0,0,0,0,0,0,0,...,NaN,0,0,0,0,0,0,0,0,0
3,France,Some college/university study without earning ...,0,0,0,1,0,0,0,1,...,"25,000-29,999",1,0,0,0,0,0,0,0,0
4,India,Bachelor’s degree,0,0,0,1,1,1,1,0,...,NaN,0,0,0,0,0,0,0,0,0


In [126]:
target = ps.BinaryTarget('Q8', 'Master’s degree')
searchspace = ps.create_selectors(filtered, ignore=['Q8'])
task = ps.SubgroupDiscoveryTask (
    filtered,
    target,
    searchspace,
    result_set_size=5,
    depth=4,
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)

/run/media/artixs/nvmekin/code/kaggle-datamining/venv/lib/python3.13/site-packages/pysubgroup/binary_target.py:356: RuntimeWarning: invalid value encountered in divide
  p_subgroup = np.divide(positives_subgroup, instances_subgroup)


In [127]:
result.to_dataframe()

,quality,subgroup,size_sg,size_dataset,positives_sg,positives_dataset,size_complement,relative_size_sg,relative_size_complement,coverage_sg,coverage_complement,target_share_sg,target_share_complement,target_share_dataset,lift
0,0.022626,Q13_Jupyter Notebook==1 AND Q13_MATLAB ==0 AND...,12059,23997,5137,9142,11938,0.502521,0.497479,0.561912,0.438088,0.425989,0.335483,0.380964,1.118186
1,0.022626,Q13_Jupyter Notebook==1 AND Q13_MATLAB ==0 AND...,12059,23997,5137,9142,11938,0.502521,0.497479,0.561912,0.438088,0.425989,0.335483,0.380964,1.118186
2,0.022626,Q13_Jupyter Notebook==1 AND Q13_MATLAB ==0 AND...,12059,23997,5137,9142,11938,0.502521,0.497479,0.561912,0.438088,0.425989,0.335483,0.380964,1.118186
3,0.022622,Q13_Jupyter Notebook==1 AND Q13_MATLAB ==0 AND...,12054,23997,5135,9142,11943,0.502313,0.497687,0.561693,0.438307,0.426000,0.335510,0.380964,1.118214
4,0.022568,Q13_Jupyter Notebook==1 AND Q13_MATLAB ==0 AND...,12039,23997,5128,9142,11958,0.501688,0.498312,0.560928,0.439072,0.425949,0.335675,0.380964,1.118081
